## Collecting Data

To collect my Spotify Data I used this source: https://github.com/plamere/spotipy/tree/master/examples

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 


client_id = '3d5be77a0a8b4b34a19fe1c509719169'
client_secret = 'b5f6d556e9a74f48a88ec4e4a40bfc5e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)

    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
        
    
    return ids



ids = getTrackIDs('melll', '7rdJiw7WQVYHbVLZVt682k')


def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  length = meta['duration_ms']
  popularity = meta['popularity']
 
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  mode = features[0]['mode']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, length, popularity, danceability, acousticness, energy, instrumentalness, mode, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track



# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
tracks_df = pd.DataFrame(tracks, columns = ['trackname', 'album', 'artist', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', ' mode', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [3]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         28 non-null     object 
 1   album             28 non-null     object 
 2   artist            28 non-null     object 
 3   length            28 non-null     int64  
 4   popularity        28 non-null     int64  
 5   danceability      28 non-null     float64
 6   acousticness      28 non-null     float64
 7   energy            28 non-null     float64
 8   instrumentalness  28 non-null     float64
 9    mode             28 non-null     int64  
 10  liveness          28 non-null     float64
 11  loudness          28 non-null     float64
 12  speechiness       28 non-null     float64
 13  tempo             28 non-null     float64
 14  time_signature    28 non-null     int64  
 15  valence           28 non-null     float64
dtypes: float64(9), int64(4), object(3)
memory usag

In [4]:
dct = sp.playlist("7rdJiw7WQVYHbVLZVt682k", fields=None, market=None, additional_types=('track', ))
result = sp.search(dct['tracks']['items'][0]['track']['artists'][0]['name'])
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

genres = artist["genres"]
artist = artist['name']
print(genres)
print (artist)

artist genres: ['german dance', 'german pop', 'partyschlager', 'schlager']
['german dance', 'german pop', 'partyschlager', 'schlager']
Stereoact


In [5]:
from tqdm import tqdm
import pandas as pd

In [6]:
rows = []
for t in tqdm(range(len(tracks_df))):
    track_row = []
    
    track = dct['tracks']['items'][t]['track']['name']
    artist = dct['tracks']['items'][t]['track']['artists'][0]['name']
    
    result = sp.search(dct['tracks']['items'][t]['track']['artists'][0]['name'])
    tr = result['tracks']['items'][0]
    art = sp.artist(tr["artists"][0]["external_urls"]["spotify"])
    
    genre = art["genres"]
    try:
        track_row = [artist, genre[0]]
    except:
        track_row = [artist]
    rows.append(track_row)
    
    
genre = pd.DataFrame.from_dict(rows)

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  2.84it/s]


In [7]:
genre.columns = ['artist', 'genres']

In [8]:
genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  28 non-null     object
 1   genres  26 non-null     object
dtypes: object(2)
memory usage: 576.0+ bytes


In [9]:
df = pd.merge(tracks_df, genre,  on = 'artist') #"artist" is the common identifier
df.head()

,trackname,album,artist,length,popularity,danceability,acousticness,energy,instrumentalness,mode,liveness,loudness,speechiness,tempo,time_signature,valence,genres
0,Die immer lacht - Radio 2016 Mix,Die immer lacht,Stereoact,210573,57,0.729,0.026200,0.542,0.000005,0,0.2110,-7.671,0.0844,127.995,4,0.520,german dance
1,Sex ohne Grund (feat. Shindy),Rumble in the Jungle,Ali Bumaye,187760,62,0.899,0.167000,0.600,0.000000,0,0.0469,-7.052,0.3270,93.982,4,0.685,gangster rap
2,Bossy Featuring Too $hort (feat. Too $hort),The Hits,Kelis,272360,41,0.906,0.000172,0.636,0.000005,1,0.0511,-5.318,0.1880,84.011,4,0.782,alt z
3,Paradise - Radio Edit,Paradise (Radio Edit),Benny Benassi,233168,46,0.592,0.009230,0.865,0.000000,1,0.1200,-5.426,0.0494,127.877,4,0.511,edm
4,Bonnie & Clyde (feat. Amir der Sänger),Ego (Premium Edition),Fard,190528,35,0.380,0.123000,0.839,0.000000,1,0.1680,-6.554,0.4100,165.642,4,0.459,persian alternative


In [10]:
grouped = df.groupby(['trackname','artist'], as_index=True).size()
grouped[grouped > 1].count()

3

In [11]:
df.drop_duplicates(subset=['trackname','artist'], inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 32
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         27 non-null     object 
 1   album             27 non-null     object 
 2   artist            27 non-null     object 
 3   length            27 non-null     int64  
 4   popularity        27 non-null     int64  
 5   danceability      27 non-null     float64
 6   acousticness      27 non-null     float64
 7   energy            27 non-null     float64
 8   instrumentalness  27 non-null     float64
 9    mode             27 non-null     int64  
 10  liveness          27 non-null     float64
 11  loudness          27 non-null     float64
 12  speechiness       27 non-null     float64
 13  tempo             27 non-null     float64
 14  time_signature    27 non-null     int64  
 15  valence           27 non-null     float64
 16  genres            25 non-null     object 
dtyp

In [13]:
df.to_csv("playlist5.csv", sep = ',')